<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250618/2%E5%A4%89%E6%95%B02%E5%88%B6%E7%B4%84%E6%9C%80%E9%81%A9%E5%8C%96%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E8%A7%A3%E8%AA%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#演習問題 解答

---

### **準備: ライブラリのインストール**
最初に、数理最適化問題を解くための`python-mip`ライブラリをインストールする必要がある。以下のコードセルを実行する。

In [ ]:
%%capture
# 演習問題の解答コードを実行する前に、このセルを一度だけ実行してください。
!pip install mip

##例題の数理モデル（再掲）

例題の数理モデルは以下のようなものであった。

$$
\begin{array}{lll}
\text{Maximize} & Z = 5x_1 + 4x_2 & (\text{目的関数}) \\
\text{Subject to} & 2x_1 + x_2 \le 10 & (\text{労働時間制約}) \\
& x_1 + 2x_2 \le 8 & (\text{原材料制約}) \\
& x_1 \ge 0 & (\text{非負条件}) \\
& x_2 \ge 0 & (\text{非負条件})
\end{array}
$$

最適化結果は

* 最大利益 ($Z$): 28.0 万円
* 最適生産計画:
   * 製品A ($x_1$): 4.0 単位
   * 製品B ($x_2$): 2.0 単位

---

## 演習問題1
**問題:**
例題の目的関数が $Z=2x_{1}+3x_{2}$ に変更された場合の最適生産計画 $(x_{1},x_{2})$ と最大利益Zを求めなさい。

### **1. 数理モデルの定式化**
本問題は、例題の生産計画問題において、製品Aと製品Bの単位あたり利益が変更されたケースである。決定変数と制約条件は例題と同一であり、目的関数のみが変更される。

* **決定変数**
    * $x_{1}$: 製品Aの生産量(単位)
    * $x_{2}$: 製品Bの生産量(単位)

* **目的関数 (変更点)**
    製品Aの利益が2万円、製品Bの利益が3万円となるため、総利益Zを最大化する目的関数は以下のように表される。
    $$
    \text{Maximize} \quad Z = 2x_{1} + 3x_{2}
    $$

* **制約条件 (例題と同様)**
    * 労働時間制約: $2x_{1} + x_{2} \le 10$
    * 原材料制約: $x_{1} + 2x_{2} \le 8$
    * 非負条件: $x_{1} \ge 0, \quad x_{2} \ge 0$

### **2. Python (MIP) による実装**

In [ ]:
# モジュールのインポート
import mip

# 1. モデルオブジェクトの作成
# 問題名を"ex1_production"、目的を最大化(MAXIMIZE)としてモデルを作成する。
model = mip.Model(name="ex1_production", sense=mip.MAXIMIZE)

# 2. 決定変数の定義
# 製品Aの生産量 x1: 連続変数、下限は0.0
x1 = model.add_var(name="x1", var_type=mip.CONTINUOUS, lb=0.0)
# 製品Bの生産量 x2: 連続変数、下限は0.0
x2 = model.add_var(name="x2", var_type=mip.CONTINUOUS, lb=0.0)

# 3. 目的関数の設定
# 変更された目的関数 Z = 2*x1 + 3*x2 を設定する。
model.objective = 2*x1 + 3*x2

# 4. 制約条件の追加
# 労働時間制約
model.add_constr(2*x1 + x2 <= 10, name="labor")
# 原材料制約
model.add_constr(x1 + 2*x2 <= 8, name="material")

# 5. 最適化の実行
status = model.optimize()

# 6. 結果の確認
print("--- 演習問題1 最適化結果 ---")
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"状態: 最適解発見 (OPTIMAL)")
    print(f"最大利益 (Z): {model.objective_value:.1f} 万円")
    print("最適生産計画:")
    print(f"  製品A (x1): {x1.x:.2f} 単位")
    print(f"  製品B (x2): {x2.x:.2f} 単位")
else:
    print(f"状態: {status}")
    print("最適解が見つかりませんでした。")

###コメント
目的関数が変更になったので、最適値（$Z$）は変化した。
このケースでは（たまたま）最適解（$x_1,x2$）は
例題と同じであった。

---

## 演習問題2
**問題:**
例題の原材料の利用可能上限が $b_{2}=12$ に増加した場合 ($x_{1}+2x_{2}\le12$ となる)、新しい最適生産計画 $(x_{1},x_{2})$ と最大利益Zを求めなさい。(他の条件は例題と同じ)

### **1. 数理モデルの定式化**
本問題では、利用可能な原材料の上限が8kgから12kgに増加する。目的関数および労働時間制約は例題と同一である。

* **決定変数**
    * $x_{1}$: 製品Aの生産量(単位)
    * $x_{2}$: 製品Bの生産量(単位)

* **目的関数 (例題と同様)**
    $$
    \text{Maximize} \quad Z = 5x_{1} + 4x_{2}
    $$

* **制約条件**
    * 労働時間制約: $2x_{1} + x_{2} \le 10$
    * 原材料制約: $x_{1} + 2x_{2} \le 12$ **(変更点)**
    * 非負条件: $x_{1} \ge 0, \quad x_{2} \ge 0$

### **2. Python (MIP) による実装**

In [ ]:
# モジュールのインポート
import mip

# 1. モデルオブジェクトの作成
model = mip.Model(name="ex2_production", sense=mip.MAXIMIZE)

# 2. 決定変数の定義
x1 = model.add_var(name="x1", var_type=mip.CONTINUOUS, lb=0.0)
x2 = model.add_var(name="x2", var_type=mip.CONTINUOUS, lb=0.0)

# 3. 目的関数の設定 (例題と同じ)
model.objective = 5*x1 + 4*x2

# 4. 制約条件の追加
# 労働時間制約
model.add_constr(2*x1 + x2 <= 10, name="labor")
# 原材料制約 (上限値を12に変更)
model.add_constr(x1 + 2*x2 <= 12, name="material")

# 5. 最適化の実行
status = model.optimize()

# 6. 結果の確認
print("--- 演習問題2 最適化結果 ---")
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"状態: 最適解発見 (OPTIMAL)")
    print(f"最大利益 (Z): {model.objective_value:.2f} 万円")
    print("最適生産計画:")
    print(f"  製品A (x1): {x1.x:.2f} 単位")
    print(f"  製品B (x2): {x2.x:.2f} 単位")
else:
    print(f"状態: {status}")
    print("最適解が見つかりませんでした。")


---

## 演習問題3
**問題:**
例題に、製品Aと製品Bは、それぞれ最低1単位は生産しなければならない、という追加制約が課された場合(すなわち $x_{1}\ge1$ かつ $x_{2}\ge1$)、最適生産計画 $(x_{1},x_{2})$ と最大利益を求めなさい。(他の条件は例題と同じ)

### **1. 数理モデルの定式化**
本問題では、各製品の生産量に下限が設けられる。これは非負条件がより厳しい制約に置き換わることを意味する。

* **決定変数**
    * $x_{1}$: 製品Aの生産量(単位)
    * $x_{2}$: 製品Bの生産量(単位)

* **目的関数 (例題と同様)**
    $$
    \text{Maximize} \quad Z = 5x_{1} + 4x_{2}
    $$

* **制約条件**
    * 労働時間制約: $2x_{1} + x_{2} \le 10$
    * 原材料制約: $x_{1} + 2x_{2} \le 8$
    * 生産量下限制約 ($変更点$): $x_{1} \ge 1, \quad x_{2} \ge 1$

### **2. Python (MIP) による実装**
この新しい制約は、決定変数を定義する際の `add_var` メソッドの下限( `lb` )パラメータを `0.0` から `1.0` に変更することで、効率的にモデル化できる。

In [ ]:
# モジュールのインポート
import mip

# 1. モデルオブジェクトの作成
model = mip.Model(name="ex3_production", sense=mip.MAXIMIZE)

# 2. 決定変数の定義
# 製品Aの生産量 x1: 下限(lb)を1.0に変更
x1 = model.add_var(name="x1", var_type=mip.CONTINUOUS, lb=1.0)
# 製品Bの生産量 x2: 下限(lb)を1.0に変更
x2 = model.add_var(name="x2", var_type=mip.CONTINUOUS, lb=1.0)

# 3. 目的関数の設定 (例題と同じ)
model.objective = 5*x1 + 4*x2

# 4. 制約条件の追加
# 労働時間制約
model.add_constr(2*x1 + x2 <= 10, name="labor")
# 原材料制約
model.add_constr(x1 + 2*x2 <= 8, name="material")

# 5. 最適化の実行
status = model.optimize()

# 6. 結果の確認
print("--- 演習問題3 最適化結果 ---")
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"状態: 最適解発見 (OPTIMAL)")
    print(f"最大利益 (Z): {model.objective_value:.1f} 万円")
    print("最適生産計画:")
    print(f"  製品A (x1): {x1.x:.2f} 単位")
    print(f"  製品B (x2): {x2.x:.2f} 単位")
else:
    print(f"状態: {status}")
    print("最適解が見つかりませんでした。")

---

## 演習問題4
**問題:**
市場の要求により、「製品Aと製品Bの合計生産量は、少なくとも5単位以上でなければならない」という制約が追加されました。この新しい問題を定式化し、最適生産計画 $(x_{1},x_{2})$ と最大利益Zを求めなさい。(他の条件は例題と同じ)

### **1. 数理モデルの定式化**
本問題は、例題のモデルに新しい制約を追加するものである。

* **決定変数**
    * $x_{1}$: 製品Aの生産量(単位)
    * $x_{2}$: 製品Bの生産量(単位)

* **目的関数 (例題と同様)**
    $$
    \text{Maximize} \quad Z = 5x_{1} + 4x_{2}
    $$

* **制約条件**
    * 労働時間制約: $2x_{1} + x_{2} \le 10$
    * 原材料制約: $x_{1} + 2x_{2} \le 8$
    * 合計生産量制約 (追加): $x_{1} + x_{2} \ge 5$
    * 非負条件: $x_{1} \ge 0, \quad x_{2} \ge 0$

### **2. Python (MIP) による実装**

In [ ]:
# モジュールのインポート
import mip

# 1. モデルオブジェクトの作成
model = mip.Model(name="ex4_production", sense=mip.MAXIMIZE)

# 2. 決定変数の定義
x1 = model.add_var(name="x1", var_type=mip.CONTINUOUS, lb=0.0)
x2 = model.add_var(name="x2", var_type=mip.CONTINUOUS, lb=0.0)

# 3. 目的関数の設定 (例題と同じ)
model.objective = 5*x1 + 4*x2

# 4. 制約条件の追加
# 労働時間制約
model.add_constr(2*x1 + x2 <= 10, name="labor")
# 原材料制約
model.add_constr(x1 + 2*x2 <= 8, name="material")
# 合計生産量制約 (追加)
model.add_constr(x1 + x2 >= 5, name="total_production")

# 5. 最適化の実行
status = model.optimize()

# 6. 結果の確認
print("--- 演習問題4 最適化結果 ---")
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"状態: 最適解発見 (OPTIMAL)")
    print(f"最大利益 (Z): {model.objective_value:.1f} 万円")
    print("最適生産計画:")
    print(f"  製品A (x1): {x1.x:.2f} 単位")
    print(f"  製品B (x2): {x2.x:.2f} 単位")
else:
    print(f"状態: {status}")
    print("最適解が見つかりませんでした。")

---

## 演習問題5
**問題:**
もし製品Aを生産する際に、段取り替えのために固定で労働時間が1時間必要になる場合(生産量が0より大きい場合のみ)、この状況をモデルに組み込み、最適生産計画 $(x_{1},x_{2})$ と最大利益Zを求める問題を定式化し、解を求めなさい。

### **1. 数理モデルの定式化**
この問題は、生産量に依存しない固定の資源消費をモデル化するため、0か1の値を取る**バイナリ変数**を導入する必要がある。これにより、問題は線形計画問題から**混合整数計画(MIP)問題**となる。

* **決定変数**
    * $x_{1}$: 製品Aの生産量(単位)
    * $x_{2}$: 製品Bの生産量(単位)
    * $y_{1}$: 製品Aを生産するかどうかを示すバイナリ変数 ($1$なら生産する, $0$なら生産しない)

* **目的関数 (例題と同様)**
    $$
    \text{Maximize} \quad Z = 5x_{1} + 4x_{2}
    $$

* **制約条件**
    * **労働時間制約(変更)**: 製品Aを生産する場合($y_1=1$)にのみ、固定で1時間の労働時間が消費される。
        $$
        2x_{1} + x_{2} + 1y_{1} \le 10
        $$
    * **原材料制約**:
        $$
        x_{1} + 2x_{2} \le 8
        $$
    * **Big-M制約**: バイナリ変数 $y_1$ と連続変数 $x_1$ を関連付けるための制約。$x_1$ が（0でなく）正となれば$y_1$が1となる制約を追加する。
        $$
        x_{1} \le M \cdot y_{1}
        $$
        ここで $M$ は、$x_1$ が取りうる最大値よりも十分に大きい定数とする（Big-M と呼ばれる）。
        
        この条件より $y_1=0$ の場合は $x_1 \le 0 $ より $x_1=0$ となる。
        
        また、
        $x_1>0$ であれば、$y_1=1$ となる。
        なお、$M$は、他の制約から $x_1$ は最大でも5 ($2x_1 \le 10$) を超えないため、$M=5$ と設定すれば十分である。
    * **変数条件**:
        $$
        x_{1} \ge 0, \quad x_{2} \ge 0, \quad y_{1} \in \{0, 1\}
        $$

### **2. Python (MIP) による実装**

In [ ]:
# モジュールのインポート
import mip

# 1. モデルオブジェクトの作成
model = mip.Model(name="ex5_production_mip", sense=mip.MAXIMIZE)

# 2. 決定変数の定義
# 連続変数
x1 = model.add_var(name="x1", var_type=mip.CONTINUOUS, lb=0.0)
x2 = model.add_var(name="x2", var_type=mip.CONTINUOUS, lb=0.0)
# バイナリ変数 (製品Aを生産するか否か)
y1 = model.add_var(name="y1", var_type=mip.BINARY)

# 3. 目的関数の設定 (例題と同じ)
model.objective = 5*x1 + 4*x2

# 4. 制約条件の追加
# 労働時間制約 (固定消費分を追加)
model.add_constr(2*x1 + x2 + y1 <= 10, name="labor_modified")
# 原材料制約
model.add_constr(x1 + 2*x2 <= 8, name="material")
# Big-M制約 (M=5とする)
M = 5.0
model.add_constr(x1 <= M * y1, name="big_m")

# 5. 最適化の実行
status = model.optimize()

# 6. 結果の確認
print("--- 演習問題5 最適化結果 ---")
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"状態: 最適解発見 (OPTIMAL)")
    print(f"最大利益 (Z): {model.objective_value:.1f} 万円")
    print("最適生産計画:")
    print(f"  製品A (x1): {x1.x:.2f} 単位")
    print(f"  製品B (x2): {x2.x:.2f} 単位")
    print(f"  製品A生産有無 (y1): {'生産する' if y1.x > 0.5 else '生産しない'} ({y1.x:.0f})")
else:
    print(f"状態: {status}")
    print("最適解が見つかりませんでした。")